In [1]:
import urllib
import bs4 
import numpy as np
import pandas as pd
import re
import csv
import matplotlib.pyplot as plt

# Scraping Date, Start Time, Attendance and Venue from Boxscore

In [1]:
def parse_boxscore(book_link,year):
    book_page = urllib.request.urlopen(book_link).read()
    soup = bs4.BeautifulSoup(book_page, 'html.parser')
    author = soup.find('div', class_= "scorebox_meta")
    
    # Webpage boxscore structures are different for years after 2018
    if year<2018: 
        date=author.contents[1].string
        time=author.contents[3].string
        attendance=author.contents[5].text
        venue=author.contents[7].text
    else:
        date=author.contents[1].string
        time=author.contents[2].string
        attendance=author.contents[3].text
        venue=author.contents[4].text
            
    mlb = [date, time, attendance, venue]
    return mlb

# Grabbing All Links from Webpage

In [2]:
def extract_link_one_page(year):
    domain_name='https://www.baseball-reference.com'
    url = domain_name+ "/teams/TOR/" + str(year) + "-schedule-scores.shtml"
    html_page = urllib.request.urlopen(url).read()
    soup = bs4.BeautifulSoup(html_page, 'html.parser')
    links_of_one_page = [domain_name + td.find('a').get("href") for td in soup.find_all('td',{'data-stat':'boxscore'})]
    return links_of_one_page

# Scraping Through all Links on MLB Schedule Page

In [3]:
column_names = ['date', 'time', 'attendance','venue']

def scrape_mlb(end_year, export_csv_file, start_year):
    with open(export_csv_file, 'a') as write_obj:
        writer = csv.writer(write_obj)
        writer.writerow(column_names)
    for yr in np.arange(start_year, (end_year + 1)):
        print("Processing year " + str(yr) + "...")
        with open(export_csv_file, 'a') as write_obj:
            writer = csv.writer(write_obj)
            links_of_one_page = extract_link_one_page(yr)
            print(yr)
            i = 0
            for link in links_of_one_page:
                book_info = parse_boxscore(link,yr)
                i += 1
                if i % 10 == 0:
                    print(i)
                print(link)
                print(book_info)
                writer.writerow(book_info)

#export_csv_file = "/Users/tung-linwu/Desktop/Insight/data/mlb_scrape/mlb_times.csv"
#scrape_books = scrape_books(2018,export_csv_file, 2014)